# Emission Models

This notebook demonstrates how to construct emission models with `cherab.imas`-loaded plasmas.

Prerequisites: [Pooch](https://www.fatiando.org/pooch/) must be installed to download the example data.

In [ ]:
import numpy as np
import ultraplot as uplt
from raysect.core.math import Point3D, Vector3D
from raysect.core.workflow import MulticoreEngine
from raysect.optical import Ray, Spectrum, World
from rich import print as rprint
from rich.table import Table

from cherab.core import Line, elements
from cherab.core.math import sample3d, sample3d_grid
from cherab.core.model import Bremsstrahlung, ExcitationLine, RecombinationLine
from cherab.imas.datasets import iter_jintrac
from cherab.imas.plasma import load_plasma

# Set dark background for plots
uplt.rc.style = "dark_background"

## Load full plasma from IMAS

The details of loading a full plasma are covered in the [](./full_plasma.ipynb) notebook.
Here we simply load a plasma from an IMAS jintrac dataset.

In [ ]:
world = World()
path = iter_jintrac()
plasma = load_plasma(path, "r", parent=world)

Show which species the plasma is composed of.

In [ ]:
species = sorted(
    [s for s in plasma.composition],
    key=lambda x: (x.element.symbol, x.charge),
)

table = Table(title="Plasma Species")
table.add_column("Element", style="cyan", justify="right")
table.add_column("Charge")
symbol = ""
for s in species:
    _symbol = s.element.symbol
    if _symbol != symbol:
        table.add_row(s.element.symbol, f"{s.charge:>2d}+")
        symbol = _symbol
    else:
        table.add_row("", f"{s.charge:>2d}+")
rprint(table)

## Construct emission models

### Retrieve atomic data

[OpenADAS](https://open.adas.ac.uk) provides data for a variety of emission processes in plasmas.
Here we demonstrate how to include some of these processes in a Cherab simulation and generate synthetic spectra.

In [ ]:
from cherab.openadas import OpenADAS
from cherab.openadas.repository import populate

# Download any missing ADAS data files from OpenADAS
populate()

# Set up atomic data source
plasma.atomic_data = OpenADAS(permit_extrapolation=True)

Currently, neon and tangsten atomic data files (adf15) are to be downloaded manually as a workaround.

In [ ]:
from cherab.openadas.install import install_files

install_files(
    {
        "adf15": (
            (elements.neon, 0, "adf15/pec96#ne/pec96#ne_pjr#ne0.dat"),  # pjr: metastable resolved
            (elements.neon, 1, "adf15/pec96#ne/pec96#ne_pjr#ne1.dat"),
            # TODO: fix the parser to accept these files
            # (elements.tungsten, 0, "adf15/pec40#w/pec40#w_ic#w0.dat"),
            # (elements.tungsten, 1, "adf15/pec40#w/pec40#w_ic#w1.dat"),
        )
    },
    download=True,
)

### Apply emission models to plasma species

Here we create emission models for Bremsstrahlung, excitation lines, and recombination lines for the main plasma species (D, T, He, Ne, W).
Each line emission model is manually configured with `cherab`'s `Line` class to specify the desired transition.

In [ ]:
hydrogen_like_transitions = [
    (2, 1),
    (3, 1),
    (3, 2),
    (4, 1),
    (4, 2),
    (4, 3),
    (5, 1),
    (5, 2),
    (5, 3),
    (5, 4),
    (6, 1),
    (6, 2),
    (6, 3),
    (6, 4),
    (6, 5),
    (7, 1),
    (7, 2),
    (7, 3),
    (7, 4),
    (7, 5),
    (7, 6),
    (8, 1),
    (8, 2),
    (8, 3),
    (8, 4),
    (8, 5),
    (8, 6),
    (8, 7),
    (9, 1),
    (9, 2),
    (9, 3),
    (9, 4),
    (9, 5),
    (9, 6),
    (9, 7),
    (9, 8),
    (10, 1),
    (10, 2),
    (10, 3),
    (10, 4),
    (10, 5),
    (10, 6),
    (10, 7),
    (10, 8),
    (10, 9),
    (11, 1),
    (11, 2),
    (11, 3),
    (11, 4),
    (11, 5),
    (11, 6),
    (11, 7),
    (11, 8),
    (11, 9),
    (11, 10),
    (12, 1),
    (12, 2),
    (12, 3),
    (12, 4),
    (12, 5),
    (12, 6),
    (12, 7),
    (12, 8),
    (12, 9),
    (12, 10),
    (12, 11),
]

lines = {
    "d0+": [  # D 0+
        *[Line(elements.deuterium, 0, t) for t in hydrogen_like_transitions],
    ],
    "t0+": [  # T 0+
        *[Line(elements.tritium, 0, t) for t in hydrogen_like_transitions],
    ],
    "he0+": [  # He 0+
        Line(elements.helium, 0, ("1s1 4p1 1P1.0", "1s2 1S.0")),  # 52.22 nm
        Line(elements.helium, 0, ("1s1 3p1 1P1.0", "1s2 1S.0")),  # 53.70 nm
        Line(elements.helium, 0, ("1s1 2p1 1P1.0", "1s2 1S.0")),  # 58.44 nm
        Line(elements.helium, 0, ("1s1 4p1 3P4.0", "1s1 2s1 3S1.0")),  # 318.87 nm
        Line(elements.helium, 0, ("1s1 3p1 3P4.0", "1s1 2s1 3S1.0")),  # 388.97 nm
        Line(elements.helium, 0, ("1s1 4p1 1P1.0", "1s1 2s1 1S.0")),  # 396.57 nm
        Line(elements.helium, 0, ("1s1 4d1 3D7.0", "1s1 2p1 3P4.0")),  # 447.29 nm
        Line(elements.helium, 0, ("1s1 4s1 3S1.0", "1s1 2p1 3P4.0")),  # 471.48 nm
        Line(elements.helium, 0, ("1s1 4d1 1D2.0", "1s1 2p1 1P1.0")),  # 492.32 nm
        Line(elements.helium, 0, ("1s1 3p1 1P1.0", "1s1 2s1 1S.0")),  # 501.71 nm
        Line(elements.helium, 0, ("1s1 4s1 1S.0", "1s1 2p1 1P1.0")),  # 504.90 nm
        Line(elements.helium, 0, ("1s1 3d1 3D7.0", "1s1 2p1 3P4.0")),  # 587.75 nm
        Line(elements.helium, 0, ("1s1 3d1 1D2.0", "1s1 2p1 1P1.0")),  # 668.00 nm
        Line(elements.helium, 0, ("1s1 3s1 3S1.0", "1s1 2p1 3P4.0")),  # 706.76 nm
        Line(elements.helium, 0, ("1s1 3s1 1S.0", "1s1 2p1 1P1.0")),  # 728.33 nm
    ],
    "he1+": [  # He 1+
        Line(elements.helium, 1, (2, 1)),
        Line(elements.helium, 1, (3, 1)),
        Line(elements.helium, 1, (3, 2)),
        Line(elements.helium, 1, (4, 1)),
        Line(elements.helium, 1, (4, 2)),
        Line(elements.helium, 1, (4, 3)),
        Line(elements.helium, 1, (5, 1)),
        Line(elements.helium, 1, (5, 2)),
        Line(elements.helium, 1, (5, 3)),
    ],
    "ne0+": [  # Ne 0+
        Line(elements.neon, 0, ("2s2 2p5 3d1 3D7.0", "2s2 2p6 1S0.0")),  # 61.69 nm
        Line(elements.neon, 0, ("2s2 2p5 3d1 1P1.0", "2s2 2p6 1S0.0")),  # 61.87 nm
        Line(elements.neon, 0, ("2s2 2p5 3d1 3P4.0", "2s2 2p6 1S0.0")),  # 61.89 nm
        Line(elements.neon, 0, ("2s2 2p5 3s1 1P1.0", "2s2 2p6 1S0.0")),  # 73.59 nm
        Line(elements.neon, 0, ("2s2 2p5 3p1 1S0.0", "2s2 2p5 3s1 3P4.0")),  # 600.59 nm
        Line(elements.neon, 0, ("2s2 2p5 3p1 1D2.0", "2s2 2p5 3s1 3P4.0")),  # 602.73 nm
        Line(elements.neon, 0, ("2s2 2p5 3p1 1P1.0", "2s2 2p5 3s1 3P4.0")),  # 605.88 nm
        Line(elements.neon, 0, ("2s2 2p5 3p1 3D7.0", "2s2 2p5 3s1 3P4.0")),  # 643.56 nm
        Line(elements.neon, 0, ("2s2 2p5 3p1 1S0.0", "2s2 2p5 3s1 1P1.0")),  # 665.39 nm
        Line(elements.neon, 0, ("2s2 2p5 3p1 1D2.0", "2s2 2p5 3s1 1P1.0")),  # 668.01 nm
        Line(elements.neon, 0, ("2s2 2p5 3p1 3P4.0", "2s2 2p5 3s1 1P1.0")),  # 668.32 nm
        Line(elements.neon, 0, ("2s2 2p5 3p1 1P1.0", "2s2 2p5 3s1 1P1.0")),  # 671.89 nm
        Line(elements.neon, 0, ("2s2 2p5 3p1 3S1.0", "2s2 2p5 3s1 3P4.0")),  # 714.77 nm
        Line(elements.neon, 0, ("2s2 2p5 3p1 3D7.0", "2s2 2p5 3s1 1P1.0")),  # 718.55 nm
        Line(elements.neon, 0, ("2s2 2p5 3d1 3D7.0", "2s2 2p5 3p1 3S1.0")),  # 723.07 nm
        Line(elements.neon, 0, ("2s2 2p5 3d1 1P1.0", "2s2 2p5 3p1 3S1.0")),  # 747.45 nm
        Line(elements.neon, 0, ("2s2 2p5 3d1 3P4.0", "2s2 2p5 3p1 3S1.0")),  # 751.26 nm
    ],
    "ne1+": [  # Ne 1+
        Line(elements.neon, 1, ("2s2 2p4 3d1 2D4.5", "2s2 2p5 2P2.5")),  # 30.52 nm
        Line(elements.neon, 1, ("2s2 2p4 3d1 2F6.5", "2s2 2p5 2P2.5")),  # 32.63 nm
        Line(elements.neon, 1, ("2s2 2p4 3d1 2D4.5", "2s2 2p5 2P2.5")),  # 32.68 nm
        Line(elements.neon, 1, ("2s2 2p4 3d1 2S0.5", "2s2 2p5 2P2.5")),  # 32.71 nm
        Line(elements.neon, 1, ("2s2 2p4 3d1 2P2.5", "2s2 2p5 2P2.5")),  # 32.75 nm
        Line(elements.neon, 1, ("2s2 2p4 3d1 4P5.5", "2s2 2p5 2P2.5")),  # 35.61 nm
        Line(elements.neon, 1, ("2s2 2p4 3d1 2F6.5", "2s2 2p5 2P2.5")),  # 35.64 nm
        Line(elements.neon, 1, ("2s2 2p4 3d1 4F13.5", "2s2 2p5 2P2.5")),  # 35.69 nm
        Line(elements.neon, 1, ("2s2 2p4 3d1 2D4.5", "2s2 2p5 2P2.5")),  # 35.70 nm
        Line(elements.neon, 1, ("2s2 2p4 3d1 4D9.5", "2s2 2p5 2P2.5")),  # 35.85 nm
        Line(elements.neon, 1, ("2s2 2p4 3s1 2S0.5", "2s2 2p5 2P2.5")),  # 36.18 nm
        Line(elements.neon, 1, ("2s2 2p4 3s1 2D4.5", "2s2 2p5 2P2.5")),  # 40.63 nm
        Line(elements.neon, 1, ("2s2 2p4 3s1 2P2.5", "2s2 2p5 2P2.5")),  # 44.64 nm
        Line(elements.neon, 1, ("2s1 2p6 2S0.5", "2s2 2p5 2P2.5")),  # 46.13 nm
        Line(elements.neon, 1, ("2s2 2p4 3p1 2P2.5", "2s2 2p4 3s1 2P2.5")),  # 123.29 nm
        Line(elements.neon, 1, ("2s2 2p4 3p1 2P2.5", "2s2 2p4 3s1 2D4.5")),  # 169.47 nm
        Line(elements.neon, 1, ("2s2 2p4 3p1 2P2.5", "2s2 2p4 3s1 4P5.5")),  # 175.70 nm
        Line(elements.neon, 1, ("2s2 2p4 3p1 2D4.5", "2s2 2p4 3s1 2P2.5")),  # 188.52 nm
        Line(elements.neon, 1, ("2s2 2p4 3p1 2P2.5", "2s2 2p4 3s1 2P2.5")),  # 192.07 nm
        Line(elements.neon, 1, ("2s2 2p4 3d1 2P2.5", "2s2 2p4 3p1 4P5.5")),  # 283.63 nm
        Line(elements.neon, 1, ("2s2 2p4 3p1 2P2.5", "2s2 2p4 3s1 4P5.5")),  # 287.62 nm
        Line(elements.neon, 1, ("2s2 2p4 3d1 4P5.5", "2s2 2p4 3p1 4P5.5")),  # 288.11 nm
        Line(elements.neon, 1, ("2s2 2p4 3d1 2F6.5", "2s2 2p4 3p1 4P5.5")),  # 289.98 nm
        Line(elements.neon, 1, ("2s2 2p4 3d1 4F13.5", "2s2 2p4 3p1 4P5.5")),  # 293.18 nm
        Line(elements.neon, 1, ("2s2 2p4 3d1 2D4.5", "2s2 2p4 3p1 4P5.5")),  # 293.97 nm
        Line(elements.neon, 1, ("2s2 2p4 3p1 4S1.5", "2s2 2p4 3s1 4P5.5")),  # 298.38 nm
        Line(elements.neon, 1, ("2s2 2p4 3d1 4D9.5", "2s2 2p4 3p1 4P5.5")),  # 304.01 nm
        Line(elements.neon, 1, ("2s2 2p4 3d1 2P2.5", "2s2 2p4 3p1 4D9.5")),  # 310.65 nm
        Line(elements.neon, 1, ("2s2 2p4 3p1 2D4.5", "2s2 2p4 3s1 4P5.5")),  # 314.70 nm
        Line(elements.neon, 1, ("2s2 2p4 3d1 4P5.5", "2s2 2p4 3p1 4D9.5")),  # 316.02 nm
        Line(elements.neon, 1, ("2s2 2p4 3d1 2F6.5", "2s2 2p4 3p1 4D9.5")),  # 318.07 nm
        Line(elements.neon, 1, ("2s2 2p4 3d1 4F13.5", "2s2 2p4 3p1 4D9.5")),  # 322.14 nm
        Line(elements.neon, 1, ("2s2 2p4 3d1 2D4.5", "2s2 2p4 3p1 4D9.5")),  # 323.10 nm
        Line(elements.neon, 1, ("2s2 2p4 3p1 2D4.5", "2s2 2p4 3s1 2D4.5")),  # 323.19 nm
        Line(elements.neon, 1, ("2s2 2p4 3d1 2P2.5", "2s2 2p4 3p1 2D4.5")),  # 329.21 nm
        Line(elements.neon, 1, ("2s2 2p4 3s1 2S0.5", "2s2 2p4 3p1 4P5.5")),  # 329.57 nm
        Line(elements.neon, 1, ("2s2 2p4 3p1 4D9.5", "2s2 2p4 3s1 4P5.5")),  # 333.77 nm
        Line(elements.neon, 1, ("2s2 2p4 3p1 2P2.5", "2s2 2p4 3s1 2D4.5")),  # 333.78 nm
        Line(elements.neon, 1, ("2s2 2p4 3p1 2P2.5", "2s2 2p4 3s1 2P2.5")),  # 334.27 nm
        Line(elements.neon, 1, ("2s2 2p4 3d1 4D9.5", "2s2 2p4 3p1 4D9.5")),  # 335.26 nm
        Line(elements.neon, 1, ("2s2 2p4 3d1 4P5.5", "2s2 2p4 3p1 2D4.5")),  # 335.26 nm
        Line(elements.neon, 1, ("2s2 2p4 3d1 2F6.5", "2s2 2p4 3p1 2D4.5")),  # 337.56 nm
        Line(elements.neon, 1, ("2s2 2p4 3d1 4F13.5", "2s2 2p4 3p1 2D4.5")),  # 342.15 nm
        Line(elements.neon, 1, ("2s2 2p4 3d1 2D4.5", "2s2 2p4 3p1 2D4.5")),  # 343.23 nm
        Line(elements.neon, 1, ("2s2 2p4 3p1 2S0.5", "2s2 2p4 3s1 2P2.5")),  # 350.79 nm
        Line(elements.neon, 1, ("2s2 2p4 3d1 4D9.5", "2s2 2p4 3p1 2D4.5")),  # 356.99 nm
        Line(elements.neon, 1, ("2s2 2p4 3p1 2F6.5", "2s2 2p4 3s1 2D4.5")),  # 357.21 nm
        Line(elements.neon, 1, ("2s2 2p4 3s1 2S0.5", "2s2 2p4 3p1 4D9.5")),  # 366.61 nm
        Line(elements.neon, 1, ("2s2 2p4 3p1 2D4.5", "2s2 2p4 3s1 2P2.5")),  # 371.41 nm
        Line(elements.neon, 1, ("2s2 2p4 3p1 4P5.5", "2s2 2p4 3s1 4P5.5")),  # 371.82 nm
        Line(elements.neon, 1, ("2s2 2p4 3p1 4D9.5", "2s2 2p4 3s1 2P2.5")),  # 398.26 nm
    ],
}

Apply the Bremsstrahlung and line emission models to the plasma species.

In [ ]:
plasma.models = [
    Bremsstrahlung(),
    *[ExcitationLine(line) for line_list in lines.values() for line in line_list],
    *[
        RecombinationLine(line)
        for line in lines["d0+"] + lines["t0+"] + lines["he0+"] + lines["he1+"] + lines["ne0+"]
    ],
]

### Visualize emission profiles

Let us visualize the total emission from all processes on a 2D grid in the R-Z plane.
Due to the heavy computational load, we limit the grid resolution to 50 mm and wavelength range to
0.0125 nm - 1240 nm with 100 bins evenly spaced in log-scale.

In [ ]:
# Grid specification
R_MIN, R_MAX = 4.0, 8.5
Z_MIN, Z_MAX = -4.5, 4.6
RES = 50.0e-3  # resolution of grid in [m]
n_r = round((R_MAX - R_MIN) / RES) + 1
n_z = round((Z_MAX - Z_MIN) / RES) + 1
extent = (R_MIN, R_MAX, Z_MIN, Z_MAX)

# Wavelength sampling
wavelengths = np.logspace(
    np.log10(0.012),
    np.log10(1240),
    100 + 1,
)
# Define wavelength bands as list of (min, max) tuples
# NOTE: Because cherab can only handle linearly spaced wavelength bins, we define wavelength bands
# as adjacent pairs of the logarithmically spaced wavelengths so that we calculate spectrum at one
# point per band.
wavelength_bands = [
    (wavelengths[i].item(), wavelengths[i + 1].item()) for i in range(len(wavelengths) - 1)
]

Sample emission at each grid point integrating over the defined wavelength bands.

In [ ]:
class Emission2DSample:
    def __init__(self, plasma, r_pts, z_pts, wavelength_bands) -> None:
        self.plasma = plasma
        self.r_pts = r_pts
        self.z_pts = z_pts
        self.indices = list(np.ndindex(len(r_pts), len(z_pts)))
        self.wavelength_bands = wavelength_bands
        self.engine = MulticoreEngine()
        self.emission_2d = np.zeros((len(r_pts), len(z_pts)))

    def run(self) -> None:
        self.engine.run(
            self.indices,
            self._render,
            self._update,
        )

    def _render(self, idx) -> float:
        i_r, i_z = idx
        x = self.r_pts[i_r]
        z = self.z_pts[i_z]

        n_e = self.plasma.electron_distribution.density(x, 0, z)
        if n_e <= 0:
            return idx, 0.0

        total = 0.0
        for min_wl, max_wl in self.wavelength_bands:
            spectrum = Spectrum(
                min_wavelength=min_wl,
                max_wavelength=max_wl,
                bins=1,
            )
            total += self._emission(x, 0, z, spectrum)
        return idx, total

    def _update(self, packed_result) -> None:
        (i_r, i_z), value = packed_result
        self.emission_2d[i_r, i_z] = value

    def _emission(self, x, y, z, spectrum: Spectrum) -> float:
        """Calculate total emission at a point for a given spectrum."""
        v = 0.0
        for model in self.plasma.models:
            v += model.emission(Point3D(x, y, z), Vector3D(0, 1, 0), spectrum).total()
        return v


# Get R, Z sampling points
r_pts, _, z_pts, _ = sample3d(
    lambda x, y, z: 0,
    (R_MIN, R_MAX, n_r),
    (0, 0, 1),
    (Z_MIN, Z_MAX, n_z),
)

# Create emission sampler and run
emission_sampler = Emission2DSample(plasma, r_pts, z_pts, wavelength_bands)
emission_sampler.run()

Plot the 2-D emission map

In [ ]:
emission_2d = emission_sampler.emission_2d
emission_2d[emission_2d <= 0] = np.nan  # for log plotting

fig, ax = uplt.subplots()
ax.imshow(
    emission_2d.T,
    discrete=False,
    extent=extent,
    origin="lower",
    norm="log",
    cmap="rocket",
    colorbar="r",
    colorbar_kw=dict(
        tickdir="out",
        formatter="log",
        label="[W/m³/sr]",
    ),
)
ax.format(
    aspect=1,
    title=f"Emissivity integrated over {wavelengths[0]:.3f} – {wavelengths[-1]:.0f} nm",
    xlabel="$R$ [m]",
    ylabel="$Z$ [m]",
    xlocator=1,
    ylocator=1,
)

## Measure line-of-sight Spectra

Here we measure line-of-sight spectra through the above modeled emission.

In [ ]:
class MeasureLoS:
    def __init__(self, world) -> None:
        self.engine = MulticoreEngine()
        self.world = world
        self.results: list[tuple[float, float]] = []

    def run(self, rays) -> np.ndarray:
        self.engine.run(rays, self.render, self.update)
        return np.asarray(
            sorted(self.results, key=lambda x: x[0]),
        )

    def render(self, ray: Ray) -> tuple[float, float]:
        s = ray.trace(self.world)
        return s.wavelengths.item(), s.samples.item()

    def update(self, packed_result) -> None:
        x, y = packed_result
        self.results.append((x, y))


# Generate rays with defined wavelength bands
origin = Point3D(8.4, 0, 1)
endpoint = Point3D(4.6, 0, -4)
direction = origin.vector_to(endpoint).normalise()
rays = [
    Ray(
        origin=origin,
        direction=direction,
        bins=1,
        max_wavelength=band[1],
        min_wavelength=band[0],
    )
    for band in wavelength_bands
]

# Measure line of sight spectrum
measure = MeasureLoS(world)
spectra = measure.run(rays)

### Visualize measured spectra

In [ ]:
fig, ax = uplt.subplots(
    refaspect=2,
    refwidth=6,
)

ax.loglog(
    spectra[:, 0],
    spectra[:, 1],
)
ax.format(
    xlabel="Wavelength [nm]",
    ylabel="Spectral Radiance [W/m²/sr/nm]",
    yformatter="log",
    grid=True,
    gridalpha=0.3,
)

Let us visualize the ray trajectory projected onto the R-Z plane along with the measured spectra.

In [ ]:
fig, axs = uplt.subplots([1, 2, 2, 2], share=False)

# Plot 2-D emissivity map
image = axs[0].imshow(
    emission_2d.T,
    extent=extent,
    origin="lower",
    norm="log",
    cmap="rocket",
)
axs[0].colorbar(
    image,
    formatter="log",
    tickdir="out",
    loc="lr",
    orientation="vertical",
    ticklabelsize="small",
    length=5,
    frameon=False,
)

# Plot line of sight
axs[0].plot(
    [origin.x, endpoint.x],
    [origin.z, endpoint.z],
    color="C0",
    label="Line of sight",
    legend="ur",
)

axs[0].format(
    aspect=1,
    xlim=(extent[0], extent[1]),
    ylim=(extent[2], extent[3]),
    xlabel="$R$ [m]",
    ylabel="$Z$ [m]",
    ylocator=1,
    xlocator=1,
    title="Emissivity [W/m³/sr]",
    gridalpha=0.1,
)

# Plot 1-D measured spectrum
axs[1].loglog(spectra[:, 0], spectra[:, 1])
axs[1].format(
    xlim=(wavelengths.min(), wavelengths.max()),
    title="Ray-traced emission spectrum",
    xlabel="Wavelength [nm]",
    ylabel="Spectral Radiance [W/m²/sr/nm]",
    yformatter="log",
    grid=True,
    gridalpha=0.3,
)